In [1]:
from imutils import paths
import random
import shutil
import json
import os 

In [2]:
dataset_path = "data/"
subset_path = "subset/"
N = 40 # number of randomly selected subjects
n = 10 # number of subsets

if not os.path.exists(subset_path):
    os.makedirs(subset_path)
else:
    !rm -r "subset/"
    os.makedirs(subset_path)
    
labels = ["backward", "forward", "right", "left", "down", "up", "go", "stop", "on", "off", "yes", "no", 
          "learn", "follow", "zero", "one", "two", "three", "four", "five", "six", "seven", "eight", "nine", 
          "bed", "bird", "cat", "dog", "happy", "house", "read", "write", "tree", "visual", "wow"]

In [3]:
wavPaths = list(paths.list_files(dataset_path, validExts="wav"))
print(len(wavPaths))

3623


In [4]:
def accuracy(preds_path, labels):
    correct = 0
    # groundtruth and predicted 
    # labels
    y_true = []
    y_pred = []

    # opening a JSON file
    f = open(preds_path)

    # returns JSON object as 
    # a dictionary
    data = json.load(f)

    # iterating through the json list
    # and adding true and predicted labels
    for t, p in data.items():
        t = t.split('/')[-2]
        y_true.append(t)
        y_pred.append(p)
        if t == p:
            correct += 1

    # closing file
    f.close()
    
    acc = correct / len(y_true)
    return acc

In [5]:
results = []

for i in range(n):
    if not os.path.exists(subset_path):
        os.makedirs(subset_path)
    else:
        !rm -r "subset/"
        os.makedirs(subset_path)
    
    for label in labels:
        source_path = os.path.join(dataset_path, label)
        subject_ids = []
        tempPaths = list(paths.list_files(source_path, validExts="wav"))

        for tempPath in tempPaths:
            wav_file = tempPath.split('/')[-1]
            subject_id = wav_file.split('.')[0]
            subject_ids.append(subject_id)
    
        random.shuffle(subject_ids)
        some_subjects = subject_ids[:N]
        
        target_path = os.path.join(subset_path, label)
        if not os.path.exists(target_path):
            os.makedirs(target_path)
        
        for some_subject in some_subjects:
            source_wav = os.path.join(source_path, some_subject+".wav")
            target_wav = os.path.join(target_path, some_subject+".wav")
            shutil.copy(source_wav, target_wav)
            
    !python inference.py --conf configs/kwmlp_ksc_tts.yaml --ckpt runs/kw-mlp-0.2.0-ksc-tts/best.pth --inp subset/ --out outputs/subset/ --lmap label_map.json --device cpu --batch_size 32
    acc = accuracy('outputs/subset/preds.json', labels)
    print("It: {}, Acc: {}".format(i, acc))
    results.append(acc)

100%|███████████████████████████████████████████| 44/44 [00:06<00:00,  6.49it/s]
Saved preds to outputs/subset/preds.json
It: 0, Acc: 0.9014285714285715
100%|███████████████████████████████████████████| 44/44 [00:06<00:00,  6.47it/s]
Saved preds to outputs/subset/preds.json
It: 1, Acc: 0.8914285714285715
100%|███████████████████████████████████████████| 44/44 [00:06<00:00,  6.42it/s]
Saved preds to outputs/subset/preds.json
It: 2, Acc: 0.8935714285714286
100%|███████████████████████████████████████████| 44/44 [00:06<00:00,  6.50it/s]
Saved preds to outputs/subset/preds.json
It: 3, Acc: 0.8978571428571429
100%|███████████████████████████████████████████| 44/44 [00:06<00:00,  6.52it/s]
Saved preds to outputs/subset/preds.json
It: 4, Acc: 0.8978571428571429
100%|███████████████████████████████████████████| 44/44 [00:06<00:00,  6.51it/s]
Saved preds to outputs/subset/preds.json
It: 5, Acc: 0.8864285714285715
100%|███████████████████████████████████████████| 44/44 [00:06<00:00,  6.40it/s]
S

In [6]:
avg_accuracy = sum(results) / len(results)
avg_accuracy

0.8956428571428573

In [7]:
results_round = ['%.4f' % result for result in results]
print(results_round)

['0.9014', '0.8914', '0.8936', '0.8979', '0.8979', '0.8864', '0.8950', '0.8957', '0.9007', '0.8964']
